# Week 6 - Homework

# Q1. Refactoring

You can see the modification made to the code in the `batch.py` file.

The if statement is the following one :  `if __name__ == "__main__":`

We run the code for February 2021 with the following command :

In [1]:
!python batch.py 2021 02

predicted mean duration: 16.191691679979066


# Q2. Installing pytest


**The second file needed is `__init__.py`**

# Q3. Writing first unit test


We write our test `test_batch.py` and we run it:

In [1]:
!pytest tests

============================= test session starts ==============================
platform linux -- Python 3.9.12, pytest-7.1.1, pluggy-1.0.0
rootdir: /home/wld/notebooks/mlops-zoomcamp/week6-best-practices/homework
plugins: anyio-3.5.0
collected 1 item                                                               

tests/test_batch.py .                                                    [100%]

============================== 1 passed in 1.45s ===============================


**There should be 2 rows in our output dataframe.**

# Q4. Mocking S3 with Localstack


With localstack, we create a s3 bucket with the following command :

In [3]:
!aws --endpoint-url=http://localhost:4566 s3 mb s3://nyc-duration

make_bucket: nyc-duration


We can check if the bucket was succesfully created :

In [4]:
!aws --endpoint-url=http://localhost:4566 s3 ls

2022-07-24 18:22:33 nyc-duration


### Make input and output paths configurable


We modified the `batch.py` file to make the input path and output path as env variable

### Reading from Localstack S3 with Pandas


We modified the `read_dataframe` function to make it read file from S3 endpoint if set.

# Q5. Creating test data


Let's save some data to S3.

In [7]:
from datetime import datetime
import pandas as pd

def dt(hour, minute, second=0):
    return datetime(2021, 1, 1, hour, minute, second)


data = [
    (None, None, dt(1, 2), dt(1, 10)),
    (1, 1, dt(1, 2), dt(1, 10)),
    (1, 1, dt(1, 2, 0), dt(1, 2, 50)),
    (1, 1, dt(1, 2, 0), dt(2, 2, 1)),        
]

columns = ['PUlocationID', 'DOlocationID', 'pickup_datetime', 'dropOff_datetime']
df = pd.DataFrame(data, columns=columns)
df

,PUlocationID,DOlocationID,pickup_datetime,dropOff_datetime
0,NaN,NaN,2021-01-01 01:02:00,2021-01-01 01:10:00
1,1.0,1.0,2021-01-01 01:02:00,2021-01-01 01:10:00
2,1.0,1.0,2021-01-01 01:02:00,2021-01-01 01:02:50
3,1.0,1.0,2021-01-01 01:02:00,2021-01-01 02:02:01


In [22]:
!export S3_ENDPOINT_URL=http://localhost:4566

In [25]:
import os

input_file = "s3://nyc-duration/in/2021-01.parquet"
S3_ENDPOINT_URL = os.getenv("S3_ENDPOINT_URL", "http://localhost:4566")


options = {
    'client_kwargs': {
        'endpoint_url': S3_ENDPOINT_URL
    }
}

df.to_parquet(
    input_file,
    engine='pyarrow',
    compression=None,
    index=False,
    storage_options=options
)

We can check if the data we added exists :

In [28]:
!awslocal s3 ls s3://nyc-duration/in --recursive --human-readable --summarize

2022-07-24 19:59:13    3.4 KiB in/2021-01.parquet

Total Objects: 1
   Total Size: 3.4 KiB


We can check the size of the file :

Size of the file : **3.4 KiB**

# Q6. Finish the integration test


We can run `main.py` to see what is the mean predicted duration with the df we just created :

In [29]:
!python batch.py 2021 01

predicted mean duration: 34.64434841620357


The mean duration is **34.64** minutes.